In [9]:
# Random forest model with gaze angles

import numpy as np
from scipy.stats import skew,kurtosis
import pandas as pd 
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.metrics import accuracy_score, classification_report
from scipy.integrate import trapz 
from sklearn.model_selection import GridSearchCV

def extract_shape_features(data):
    features = []
    for i in range(data.shape[0]):
        signal = data[i, :]
        duration = len(signal)
        peak_amplitude = np.max(signal)
        valley_amplitude = np.min(signal)
        mean_amplitude = np.mean(signal)
        time_to_peak = np.argmax(signal)
        time_to_valley = np.argmin(signal)
        zero_crossings = np.where(np.diff(np.sign(signal)))[0]
        zero_crossing_rate = len(zero_crossings) / duration
        
        signal_slope = np.gradient(signal)
        rise_time = np.where(signal >= peak_amplitude)[0][0]
        fall_time = duration - np.where(signal[::-1] >= peak_amplitude)[0][0]
        signal_curvature = np.gradient(signal_slope)
        inflection_points = np.where(np.diff(np.sign(signal_curvature)))[0]
        
        crest_factor = peak_amplitude / np.sqrt(np.mean(np.square(signal)))

        skew_val = skew(signal)
        kurt_val = kurtosis(signal)
        # Additional features (modify or add more as needed)
        std_deviation = np.std(signal)
        variance = np.var(signal)
        median = np.median(signal)
        min_to_max_ratio = valley_amplitude / peak_amplitude
        mean_abs_deviation = np.mean(np.abs(signal - mean_amplitude))
        area_under_curve = trapz(signal)
        
        # Add features to the list
        features.append([peak_amplitude, valley_amplitude, 
                         time_to_peak,time_to_valley, rise_time, fall_time,
                         len(inflection_points), crest_factor, skew_val, kurt_val,
                         std_deviation, variance, median, min_to_max_ratio, mean_abs_deviation])

    return np.array(features)

def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        eog_signal = np.loadtxt(file)
        return eog_signal
    

double = pd.read_csv('C:/Users/PC/Desktop/new_signal_double.csv')
down = pd.read_csv('C:/Users/PC/Desktop/new_signal_down.csv')
right = pd.read_csv('C:/Users/PC/Desktop/new_signal_right.csv')
left = pd.read_csv('C:/Users/PC/Desktop/new_signal_left.csv')
up = pd.read_csv('C:/Users/PC/Desktop/new_signal_up.csv')
none = pd.read_csv('C:/Users/PC/Desktop/bos_resampled.csv')

double_signal =[]
down_signal =[]
right_signal =[]
left_signal =[]
up_signal =[]
none_signal = []

double_signal.append(double)
down_signal.append(down)
right_signal.append(right)
left_signal.append(left)
up_signal.append(up)
none_signal.append(none)

size_double = np.array(double_signal)
size_down = np.array(down_signal)
size_up = np.array(up_signal)
size_left = np.array(left_signal)
size_right = np.array(right_signal)
size_none = np.array(none_signal)

size_double = size_double.reshape(size_double.shape[1],size_double.shape[2])
size_down = size_down.reshape(size_down.shape[1],size_down.shape[2])
size_up = size_up.reshape(size_up.shape[1],size_up.shape[2])
size_left = size_left.reshape(size_left.shape[1],size_left.shape[2])
size_right = size_right.reshape(size_right.shape[1],size_right.shape[2])
size_none = size_none.reshape(size_none.shape[1],size_none.shape[2])

# RIGHT FEATURES
right_gaze_angles = read_txt_file("C:/Users/PC/vscoe/right_gaze.txt")
right_features = extract_shape_features(size_right)
right_gaze_angles = np.array(right_gaze_angles)
right_features_combined = np.column_stack((right_features, right_gaze_angles))

# LEFT FEATURES
left_gaze_angles = read_txt_file("C:/Users/PC/vscoe/left_gaze.txt")
left_features = extract_shape_features(size_left)
left_gaze_angles = np.array(left_gaze_angles)
left_features_combined = np.column_stack((left_features, left_gaze_angles))

# UP FEATURES
up_gaze_angles = read_txt_file("C:/Users/PC/vscoe/up_gaze.txt")
up_features = extract_shape_features(size_up)
up_gaze_angles = np.array(up_gaze_angles)
up_features_combined = np.column_stack((up_features, up_gaze_angles))

# DOWN FEATURES
down_gaze_angles = read_txt_file("C:/Users/PC/vscoe/down_gaze.txt")
down_features = extract_shape_features(size_down)
down_gaze_angles = np.array(down_gaze_angles)
down_features_combined = np.column_stack((down_features, down_gaze_angles))

# DOUBLE FEATURES
double_gaze_angles = read_txt_file("C:/Users/PC/vscoe/double_gaze.txt")
double_features = extract_shape_features(size_double)
double_gaze_angles = np.array(double_gaze_angles)
double_features_combined = np.column_stack((double_features, double_gaze_angles))

# NONE FEATURES
none_gaze_angles = read_txt_file("C:/Users/PC/vscoe/none_gaze.txt")
none_features = extract_shape_features(size_none)
none_gaze_angles = np.array(none_gaze_angles)
none_features_combined = np.column_stack((none_features, none_gaze_angles))

all_features_combined = np.concatenate((right_features_combined, left_features_combined,up_features_combined,down_features_combined,double_features_combined,none_features_combined), axis=0)
all_features_wo_gaze = np.concatenate((right_features,left_features,up_features,down_features,double_features,none_features), axis=0)

data_order = [0] * 300 + [1] * 300 + [2] * 300 + [3] * 300 + [4] * 300 + [5] * 298
y_train = np.array(data_order)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

X_train, X_test, y_train, y_test = train_test_split(all_features_combined,y_train_encoded, test_size=0.2,random_state=42)

clf2 = RandomForestClassifier(n_estimators=400, max_depth=30,min_samples_split=2,min_samples_leaf=1,random_state=42)

clf2.fit(X_train, y_train)

y_pred = clf2.predict(X_test)

joblib.dump(clf2, 'pkl_doc/RF_model_EOG_waveform_new.pkl')

accuracy = accuracy_score(y_test, y_pred)

report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print("Random forest",report)


Accuracy: 0.9527777777777777
Random forest               precision    recall  f1-score   support

           0       0.88      0.85      0.87        53
           1       0.97      0.97      0.97        74
           2       1.00      0.98      0.99        51
           3       0.89      0.93      0.91        54
           4       0.97      1.00      0.98        63
           5       0.98      0.97      0.98        65

    accuracy                           0.95       360
   macro avg       0.95      0.95      0.95       360
weighted avg       0.95      0.95      0.95       360



'\nparam_grid = {\n    \'n_estimators\': [100, 200, 300, 400], \n    \'max_depth\': [10, 20, 30, 40],        \n    \'min_samples_split\': [2, 5, 10, 15], \n    \'min_samples_leaf\': [1, 2, 4, 8],    \n}\n\nclf = RandomForestClassifier(random_state=42)\n\ngrid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1)\n\ngrid_search.fit(X_train, y_train)\n\nbest_params = grid_search.best_params_\nbest_estimator = grid_search.best_estimator_\n\nprint("Best Parameters:", best_params)\n\nprint("Best Estimator:", best_estimator)\n\ny_pred = best_estimator.predict(X_test)\n\naccuracy = accuracy_score(y_test, y_pred)\nreport = classification_report(y_test, y_pred)\n\nprint(f\'Accuracy: {accuracy}\')\nprint("Random Forest", report)\n'

In [19]:
# make predictions with shape features

import joblib
import numpy as np
import pywt
from scipy.stats import skew, kurtosis
from scipy.integrate import trapz

def extract_shape_features(data):
    features = []
    for i in range(data.shape[0]):
        signal = data[i, :]
        duration = len(signal)
        peak_amplitude = np.max(signal)
        valley_amplitude = np.min(signal)
        mean_amplitude = np.mean(signal)
        time_to_peak = np.argmax(signal)
        zero_crossings = np.where(np.diff(np.sign(signal)))[0]
        zero_crossing_rate = len(zero_crossings) / duration

        signal_slope = np.gradient(signal)

        rise_time = np.where(signal >= peak_amplitude)[0][0]
        fall_time = duration - np.where(signal[::-1] >= peak_amplitude)[0][0]
        signal_curvature = np.gradient(signal_slope)
        inflection_points = np.where(np.diff(np.sign(signal_curvature)))[0]
        
        crest_factor = peak_amplitude / np.sqrt(np.mean(np.square(signal)))

        skew_val = skew(signal)
        kurt_val = kurtosis(signal)
        
        std_deviation = np.std(signal)
        variance = np.var(signal)
        median = np.median(signal)
        min_to_max_ratio = valley_amplitude / peak_amplitude
        mean_abs_deviation = np.mean(np.abs(signal - mean_amplitude))
        area_under_curve = trapz(signal)
        
        features.append([peak_amplitude, valley_amplitude, 
                         time_to_peak,rise_time,fall_time,
                         len(inflection_points), crest_factor, skew_val, kurt_val,
                         std_deviation, variance, median, min_to_max_ratio, mean_abs_deviation,area_under_curve])

    return np.array(features)


def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        eog_signal = np.loadtxt(file)
        return eog_signal


eog_signal = read_txt_file("C:/Users/PC/EOG/maho_txt/maho right.txt")

loaded_model = joblib.load('pkl_doc/RF_model_EOG_waveform_new.pkl')

normalized_dizi = eog_signal.reshape(1,eog_signal.shape[0])

X = extract_shape_features(normalized_dizi)
features_combined = np.column_stack((X, 44))
input_data = features_combined

predictions = loaded_model.predict(input_data)

print(predictions)



[0]
